In [19]:
# created on Dec 24, 2020
# modified on April 22, 2022
# @author:          Danell Forbes
# @email:           danellf@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [20]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [21]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [25]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "Y6XTc9QskijgE02CKTr4UaVS2"
    consumer_secret = "84maEtLYMIFuOTzUAXoKeoCl1xsQDMB1tghAqCuMblJbP905jf"
    access_token = "1517580669861253120-dHwB2bQUWFA3pbQDvThdZWMjhbF5f1"
    access_token_secret = "SKKupFnIHGTu5x0DidPNdfwpi0FaAtvs5X5iJwG2TCzB0"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    # LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
    #           -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
    #           -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii
    LOCATIONS = [-124.060542, 46.269626, -117.8925907, 48.5216548]

    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1517608283816611840, 'username': 'S_F_Warriors', 'created_at': 'Fri Apr 22 20:56:25 +0000 2022', 'lng': -122.3306025, 'lat': 47.614817, 'text': 'Jasmy on @Coinbase: https://t.co/054FdwUvAT https://t.co/ULZZMppxPo'}
{'id': 1517608331765944320, 'username': 'KellyWilsonVO', 'created_at': 'Fri Apr 22 20:56:36 +0000 2022', 'lng': -120.882482, 'lat': 47.273022, 'text': '@Its_Tina_timeVO Longest was 6 months. Shortest was a day.'}
{'id': 1517608343925256192, 'username': 'BasedSinceBirth', 'created_at': 'Fri Apr 22 20:56:39 +0000 2022', 'lng': -122.78083050000001, 'lat': 47.051444000000004, 'text': 'Hotels &gt; Airbnbs I need room service and breakfast'}
{'id': 1517608403496955904, 'username': 'Richard88227732', 'created_at': 'Fri Apr 22 20:56:53 +0000 2022', 'lng': -122.357576, 'lat': 47.094826499999996, 'text': '@TheChulaDiaries I love those earrings as well very attractive along with everything else 😮🥰😍😅'}
{'id': 1517608407053729792, 'username': 'UllerRM', 'created_at': 'Fri Apr 22 2

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
